The URL the user will go to give us info is below. This is just for beta and closed development.When the user gives us their info they will be met with localhost can't be connected to or something along the lines. For now the link is code= whatever in the url, you can copy and paste it. In the future when we get our own url, we will be able to have a function that just grabs the code without us interfering (callback function). The access token lasts for an hour and the auth code is one use. So if you used the auth code snippet once, please don't refresh it, it will give an error.

https://accounts.spotify.com/en/authorize?client_id=6b7e2e58b62d4010a958a1f14c3a2319&response_type=code&redirect_uri=http://localhost:8880/callback&scope=user-top-read%20user-read-private%20playlist-read-private

In [33]:
import re
from flask import Flask,flash, render_template, redirect, session, make_response, request, url_for
from flask_sqlalchemy import SQLAlchemy
from urllib.parse import urlencode
import os
import requests
import logging
import numpy as np
import pandas as pd
import db
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances
from pathlib import Path
from datetime import datetime
import json
from sqlalchemy import PrimaryKeyConstraint, func,Table, Column, String, ForeignKey, Boolean, PrimaryKeyConstraint, insert, or_
from sklearn.cluster import KMeans
import random
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed



# Flask setup
app = Flask(__name__)
app.static_folder = 'static'
app.secret_key = os.urandom(24)
app.config.from_pyfile('config.py')

BASE_DIR = Path(os.getcwd())
DB_PATH = BASE_DIR / 'db' / 'User.db'
app.config['SQLALCHEMY_DATABASE_URI'] = f'sqlite:///{DB_PATH}'
app.config['SQLALCHEMY_BINDS'] = {
    'two': f'sqlite:///{BASE_DIR}/Playlist.db',
    'three': f'sqlite:///{BASE_DIR}/Song.db',
    'five': f'sqlite:///{BASE_DIR}/Top_Artists.db'
}



db = SQLAlchemy(app)
playlist_song_association = db.Table('playlist_song',
    db.Column('playlist_id', db.String(255), db.ForeignKey('playlist.id'), primary_key=True),
    db.Column('song_id', db.String(255), db.ForeignKey('song.id'), primary_key=True),
    db.Column('user_id', db.String(255), db.ForeignKey('user.id'), primary_key=True),  # Adding user_id to the association table
    db.Column('added_on', db.DateTime,nullable=True)
)


# User Model
class User(db.Model):
    __tablename__ = 'user'
    id = db.Column(db.String(255), primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

# Playlist Model
class Playlist(db.Model):
    __tablename__ = 'playlist'
    id = db.Column(db.String(255), primary_key=True, nullable=False)
    name = db.Column(db.String(255))

    #can be nullable as spotify api has a "null" value for some special p[laylists]
    public = db.Column(db.Boolean)

    user_id = db.Column(db.String(255), db.ForeignKey('user.id'), nullable=False)
    user = db.relationship('User', backref=db.backref('playlists', lazy=True))
    songs = db.relationship('Song', secondary=playlist_song_association, backref=db.backref('playlists', lazy=True))


# Song Model
class Song(db.Model):
    __tablename__ = 'song'
    id = db.Column(db.String(255), primary_key=True, nullable=False)
    uid = db.Column(db.String(255), db.ForeignKey('user.id'), primary_key=True)  # Composite primary key with user ID
    name = db.Column(db.String(255))
    artists = db.Column(db.String(255))
    top_tracks = db.Column(db.Boolean, default=None)
    url = db.Column(db.String(255), nullable=True)
    genres = db.Column(db.String(255), nullable=True)
    duration = db.Column(db.Integer, nullable=True)
    __table_args__ = (
        PrimaryKeyConstraint('uid', 'id'),
        {},
    )
# SongCharacteristic Model
class SongCharacteristic(db.Model):
    __tablename__ = 'song_characteristic'
    id = db.Column(db.String(255), db.ForeignKey('song.id'), primary_key=True)
    danceability = db.Column(db.Float)
    energy = db.Column(db.Float)
    key = db.Column(db.Integer)
    loudness = db.Column(db.Float)
    mode = db.Column(db.Integer)
    speechiness = db.Column(db.Float)
    acousticness = db.Column(db.Float)
    instrumentalness = db.Column(db.Float)
    liveness = db.Column(db.Float)
    valence = db.Column(db.Float)
    tempo = db.Column(db.Float)

# Top_Artists Model
class Top_Artists(db.Model):
    __tablename__ = 'artist'
    uid = db.Column(db.String(255), primary_key=True)
    id = db.Column(db.String(255), primary_key=True, nullable=False)
    name = db.Column(db.String(255))
    image_url = db.Column(db.String(255))
    genre = db.Column(db.String(255))

# Saved_Albums Model
class Saved_Albums(db.Model):
    __tablename__ = 'saved_albums'
    uid = db.Column(db.String(255), primary_key=True)
    aid = db.Column(db.String(255), primary_key=True, nullable=False)
    name = db.Column(db.String(255))

# User_Display Model
class User_Display(db.Model):
    __tablename__ = 'user_display'
    uid = db.Column(db.String(255), primary_key=True)
    rank = db.Column(db.Integer, primary_key=True)
    song_name = db.Column(db.String(255))
    song_artist = db.Column(db.String(255))
    artist = db.Column(db.String(255))

# BlendHistory Model
class BlendHistory(db.Model):
    __tablename__ = 'blend_history'
    blend_id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    blend_name = db.Column(db.String(255), nullable=False)
    blender_id = db.Column(db.String(255), db.ForeignKey('user.id'), nullable=False)
    blende_id = db.Column(db.String(255), db.ForeignKey('user.id'), nullable=False)
    compatibility = db.Column(db.Float, nullable=True)
    blend_date = db.Column(db.DateTime, default=datetime.now)
    song_ids = db.Column(db.String, nullable=False)
    url = db.Column(db.String(255), nullable=True)
    blend_type = db.Column(db.String(100), nullable=False)
    blender = db.relationship('User', foreign_keys=[blender_id], backref=db.backref('blender_blend_history', lazy=True))
    blende = db.relationship('User', foreign_keys=[blende_id], backref=db.backref('blende_blend_history', lazy=True))
with app.app_context():
    db.create_all()
# Define constants
US_TOP_HITS_PLAYLIST_ID = "37i9dQZF1DXcBWIGoYBM5M?si=beaf719fd1f64d09"

def getToken(code):
    token_url = 'https://accounts.spotify.com/api/token'
    authorization = app.config['AUTHORIZATION']
    redirect_uri = app.config['REDIRECT_URI']
    headers = {'Authorization': f"Basic {authorization}",  # Updated the Authorization header
               'Accept': 'application/json',
               'Content-Type': 'application/x-www-form-urlencoded'}
    body = {'code': code, 'redirect_uri': redirect_uri,
            'grant_type': 'authorization_code'}
    post_response = requests.post(token_url, headers=headers, data=body)
    if post_response.status_code == 200:
        pr = post_response.json()
        return pr['access_token'], pr['refresh_token'], pr['expires_in']
    else:
        logging.error('getToken:' + str(post_response.status_code))
        return None
def get_top_tracks(user_id, access_token, limit):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/me/top/tracks?limit={limit}", headers=headers)
    
    if response.status_code == 200:
        spotify_tracks = response.json()['items']
        tracks_to_update_or_create = []

        for track in spotify_tracks:
            track_id = track['id']
            track_url = track['external_urls']['spotify']  # Get the track's URL
            duration = track['duration_ms'] // 1000  # Convert duration from milliseconds to seconds
            artists = ', '.join(artist['name'] for artist in track['artists'])
            main_artist_id = track['artists'][0]['id']

            # Fetch or create the song
            song = Song.query.filter_by(uid=user_id, id=track_id).first()
            if not song:
                # Collect artist IDs to get genres for new songs
                artist_genres = get_artist_genres_batch(access_token, [main_artist_id])
                genres = ', '.join(artist_genres.get(main_artist_id, []))

                # Create a new song record
                song = Song(
                    uid=user_id,
                    id=track_id,
                    name=track['name'],
                    artists=artists,
                    genres=genres,  # Set genres for new songs
                    top_tracks=True,  # Set top_tracks to True for new songs
                    duration=duration,  # Set duration
                    url=track_url  # Set the URL for new songs
                )
                db.session.add(song)
            else:
                # Update existing song details if necessary
                song.top_tracks = True  # Set top_tracks to True
                if song.url != track_url:
                    song.url = track_url  # Update the URL
                if song.duration != duration:
                    song.duration = duration  # Update duration

            tracks_to_update_or_create.append(song)

        # Commit the session if there were any changes
        db.session.commit()

        return tracks_to_update_or_create
    else:
        logging.error(f"get_top_tracks: {response.status_code}")
        return []


#testing saved albums

def get_saved_albums(access_token, limit=50):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/me/albums?limit={limit}", headers=headers)
    if response.status_code == 200:
        return response.json()['items']
    else:
        logging.error(f"get_saved_albums: {response.status_code}")
        return []


#testing user profile information
def get_user_profile(user_id, access_token):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/users/{user_id}", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        logging.error(f"get_user_profile: {response.status_code}")
        return []
def fetch_genres(access_token, batch_ids):
    headers = {'Authorization': f"Bearer {access_token}"}
    artist_ids_str = ','.join(batch_ids)
    response = requests.get(f"https://api.spotify.com/v1/artists?ids={artist_ids_str}", headers=headers)
    if response.status_code == 200:
        artist_data = response.json()['artists']
        return {artist['id']: artist['genres'] for artist in artist_data}
    else:
        logging.error(f"Error fetching genres: {response.status_code}")
        return {}

def get_artist_genres_batch(access_token, artist_ids):
    if not artist_ids:
        return {}

    artist_genres = {}
    batch_size = 50
    # Create a thread pool for making HTTP requests concurrently
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Submit tasks to the executor for each batch of artist IDs
        futures = [executor.submit(fetch_genres, access_token, artist_ids[i:i + batch_size]) 
                   for i in range(0, len(artist_ids), batch_size)]
        # As each task completes, update the artist_genres dictionary
        for future in as_completed(futures):
            artist_genres.update(future.result())

    return artist_genres

def get_top_artists(access_token, limit):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/me/top/artists?limit={limit}", headers=headers)
    if response.status_code == 200:
        return response.json()['items']
    else:
        logging.error(f"get_top_tracks: {response.status_code}")
        return []



def create_playlist(access_token, user_id, playlist_name, playlist_description):
    """Create a Spotify playlist for a user."""
    
    # Endpoint for creating a new playlist
    url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    
    # The headers for the HTTP request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # The payload data for the new playlist
    payload = {
        "name": playlist_name,
        "description": playlist_description,
        "public": False  # or True, if you want the playlist to be public
    }
    
    # Make the post request to create a new playlist
    response = requests.post(url, headers=headers, json=payload)
    
    # Check for successful response
    if response.status_code == 201:
        return response.json()  # Return the new playlist data as JSON
    else:
        return None  # Or handle error responses differently

def add_tracks_to_playlist(access_token, playlist_id, track_uris):
    """Add tracks to a Spotify playlist."""
    
    # Endpoint for adding tracks to a playlist
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    
    # The headers for the HTTP request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    # The payload data for the tracks to add
    payload = {
        "uris": track_uris
    }
    
    # Make the post request to add tracks
    response = requests.post(url, headers=headers, json=payload)
    
    # Check for successful response
    if response.status_code in (200, 201):
        return response.json()  # Return the response data as JSON
    else:
        return None  # Or handle error responses differently

    
def fetch_playlist_features(headers, playlist_id=US_TOP_HITS_PLAYLIST_ID):
    response = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks", headers=headers)
    playlist_tracks = response.json()
    playlist_track_ids = [track['track']['id'] for track in playlist_tracks['tracks']['items']]
    features_response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={','.join(playlist_track_ids)}", headers=headers)
    return features_response.json(), playlist_tracks
def get_user_playlists(access_token, user_id):
    headers = {'Authorization': f"Bearer {access_token}"}
    url = f"https://api.spotify.com/v1/me/playlists?limit=50"  # Start with the first page

    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            playlists_data = data['items']
            for playlist_data in playlists_data:
                playlist_id = playlist_data['id']
                # Check if playlist exists in the database
                existing_playlist = Playlist.query.filter_by(id=playlist_id).first()
                if not existing_playlist:
                    # Add the new playlist to the database
                    new_playlist = Playlist(
                        id=playlist_id,
                        name=playlist_data['name'],
                        public = playlist_data['public'],
                        user_id=user_id  # Make sure user_id is a string if your db model expects a string
                    )
                    db.session.add(new_playlist)
            db.session.commit()  # Commit after processing each page

            url = data.get('next')  # Get the URL for the next set of results
        else:
            logging.error(f"get_user_playlists: {response.status_code}")
            break  # Exit the loop if there's an error

    # Fetch playlists from the database
    db_playlists = Playlist.query.filter_by(user_id=user_id).all()
    return [{'id': playlist.id, 'name': playlist.name} for playlist in db_playlists]

def get_playlist_tracks(access_token, user_id, playlist_id):
    headers = {'Authorization': f"Bearer {access_token}"}
    playlist_response = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks", headers=headers)

    if playlist_response.status_code != 200:
        logging.error(f"get_playlist_tracks: {playlist_response.status_code}")
        return []

    tracks_data = playlist_response.json()['items']

    for track_item in tracks_data:
        track = track_item['track']
        track_id = track['id']
        main_artist_id = track['artists'][0]['id']
        track_url = track['external_urls']['spotify']

        # Fetch or create the song
        song = Song.query.filter_by(id=track_id, uid=user_id).first()
        if not song:
            # If the song does not exist in the database, fetch genres
            artist_genres = get_artist_genres_batch(access_token, [main_artist_id])
            genres = ', '.join(artist_genres.get(main_artist_id, []))

            # Create a new song record
            song = Song(
                uid=user_id,
                id=track_id,
                name=track['name'],
                artists=', '.join(artist['name'] for artist in track['artists']),
                genres=genres,
                duration=track['duration_ms'] // 1000,  # Duration in milliseconds converted to seconds
                url=track_url 
            )
            db.session.add(song)
        elif song.url != track_url:
            # Update URL if different
            song.url = track_url

        # Fetch or create the song-playlist association
        existing_association = db.session.query(playlist_song_association).filter_by(
            playlist_id=playlist_id,
            song_id=song.id,
            user_id=user_id
        ).first()

        if not existing_association:
            added_on = track_item['added_at']
            insert_stmt = insert(playlist_song_association).values(
                playlist_id=playlist_id,
                song_id=song.id,
                user_id=user_id,
                added_on=datetime.strptime(added_on, '%Y-%m-%dT%H:%M:%SZ')
            )
            db.session.execute(insert_stmt)

    db.session.commit()

    # Fetch songs from the specified playlist for the user
    db_songs = Song.query.join(
        playlist_song_association, (Song.id == playlist_song_association.c.song_id) & (Song.uid == playlist_song_association.c.user_id)
    ).filter(
        playlist_song_association.c.playlist_id == playlist_id, Song.uid == user_id
    ).all()
    
    return [{
        'id': song.id,
        'name': song.name,
        'artists': song.artists,
        'genres': song.genres,
        'url': song.url,
        'duration': song.duration
    } for song in db_songs]

def fetch_song_characteristics_batch(access_token, song_ids):
    """
    Fetch music characteristics for a batch of songs. It first checks the database,
    and if not found, it fetches from the Spotify API.

    :param access_token: Spotify API access token.
    :param song_ids: List of song IDs.
    :return: Dictionary with audio features for each song.
    """
    if not song_ids:
        return {}

    characteristics_batch = {}
    ids_to_fetch = []

    # Check the database first
    for song_id in song_ids:
        existing_characteristics = SongCharacteristic.query.filter_by(id=song_id).first()
        if existing_characteristics:
            # If found in the database, use these characteristics
            characteristics_batch[song_id] = {
                'id': existing_characteristics.id,
                'danceability': existing_characteristics.danceability,
                'energy': existing_characteristics.energy,
                'key': existing_characteristics.key,
                'loudness': existing_characteristics.loudness,
                'mode': existing_characteristics.mode,
                'speechiness': existing_characteristics.speechiness,
                'acousticness': existing_characteristics.acousticness,
                'instrumentalness': existing_characteristics.instrumentalness,
                'liveness': existing_characteristics.liveness,
                'valence': existing_characteristics.valence,
                'tempo': existing_characteristics.tempo
            }
        else:
            # If not in the database, prepare to fetch from the API
            ids_to_fetch.append(song_id)

    # Fetch from the Spotify API for IDs not found in the database
    batch_size = 50
    for i in range(0, len(ids_to_fetch), batch_size):
        batch_ids = ids_to_fetch[i:i + batch_size]
        song_ids_str = ','.join(batch_ids)
        headers = {'Authorization': f"Bearer {access_token}"}
        response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={song_ids_str}", headers=headers)

        if response.status_code == 200:
            for item in response.json()['audio_features']:
                if item:  # Ensure item is not None
                    characteristics_batch[item['id']] = item

                    # Optionally update the database
                    update_song_characteristics_in_db(item)
        else:
            logging.error(f"Batch fetch_song_characteristics for batch {i} - {i + batch_size}: {response.status_code}")

    return characteristics_batch

def update_song_characteristics_in_db(characteristics):
    """
    Update the database with new song characteristics.

    :param characteristics: Dictionary of characteristics for a single song.
    """
    new_characteristics = SongCharacteristic(
        id=characteristics['id'],
        danceability=characteristics['danceability'],
        energy=characteristics['energy'],
        key=characteristics['key'],
        loudness=characteristics['loudness'],
        mode=characteristics['mode'],
        speechiness=characteristics['speechiness'],
        acousticness=characteristics['acousticness'],
        instrumentalness=characteristics['instrumentalness'],
        liveness=characteristics['liveness'],
        valence=characteristics['valence'],
        tempo=characteristics['tempo']
    )
    db.session.add(new_characteristics)
    db.session.commit()


def fetch_all_tracks_for_user(user_id, access_token, session, is_reshuffle=False):
    # Check if the user has fetched tracks before and how many
    user_info = session.get(user_id, {'tracks_fetched': 0, 'used_playlists': []})

    # Determine the number of tracks to fetch based on reshuffle flag
    if is_reshuffle:
        tracks_to_fetch = 50
    else:
        tracks_to_fetch = 200 if user_info['tracks_fetched'] == 0 else 100

    # Fetch all playlists for the user
    playlists = get_user_playlists(access_token, user_id)

    # Filter out playlists that have been used already
    new_playlists = [pl for pl in playlists if pl['id'] not in user_info['used_playlists']]

    all_tracks = []

    for playlist in new_playlists:
        # Fetch tracks for the current playlist
        playlist_tracks = get_playlist_tracks(access_token, user_id, playlist['id'])

        # Add tracks to the all_tracks list
        all_tracks.extend(playlist_tracks)

        # Update the session info
        user_info['used_playlists'].append(playlist['id'])

        if len(all_tracks) >= tracks_to_fetch:
            break

    # Update tracks fetched count
    user_info['tracks_fetched'] += len(all_tracks)

    # Save updated user info back to the session
    session[user_id] = user_info

    return all_tracks



def get_k_nearest_songs(df_my_tracks, df_us_top_hits, selected_features, min_songs=None, max_songs=None):
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import NearestNeighbors
    import numpy as np

    # Preprocess the data
    scaler = StandardScaler().fit(df_my_tracks[selected_features])
    X_my_tracks = scaler.transform(df_my_tracks[selected_features])
    X_us_top_hits = scaler.transform(df_us_top_hits[selected_features])

    # Fit the KNN model
    knn = NearestNeighbors(n_neighbors=5, metric='cosine').fit(X_us_top_hits)
    distances, indices = knn.kneighbors(X_my_tracks)

    # Flatten the arrays and get unique song indices
    flat_distances = distances.flatten()
    flat_indices = indices.flatten()

    # Sort the distances and indices together
    sorted_recommendations = sorted(zip(flat_distances, flat_indices), key=lambda x: x[0])
    
    # If a maximum number of songs is specified, get the top closest songs
    if max_songs is not None:
        sorted_recommendations = sorted_recommendations[:max_songs]

    # If a minimum number of songs is specified, ensure at least that many unique songs
    unique_song_indices = []
    for dist, idx in sorted_recommendations:
        if idx not in unique_song_indices:
            unique_song_indices.append(idx)
        if min_songs is not None and len(unique_song_indices) >= min_songs:
            break

    # Extract the recommended unique songs from the US top hits dataframe
    recommended_songs = df_us_top_hits.iloc[unique_song_indices]
    return recommended_songs

def get_user_top_genre(user_id):
    # Query to get all genres for the given user
    genres_query = Top_Artists.query.with_entities(Top_Artists.genre).filter_by(uid=user_id).all()

    # Flatten the list of tuples and split genres into individual genres
    genres = [genre for (genre_list,) in genres_query for genre in genre_list.split(', ')]

    # Count the frequency of each genre
    genre_counts = Counter(genres)

    # Find the most common genre
    top_genre = genre_counts.most_common(1)

    if top_genre:
        return top_genre[0][0]  # Returns the most common genre
    else:
        return None  # Returns None if no genres are found
    
def blend_playlists(df_playlist1, df_playlist2, selected_features, min_songs=None):
    # Combine the two playlists
    combined_df = pd.concat([df_playlist1, df_playlist2]).reset_index(drop=True)

    # Normalize the features of the combined dataframe
    scaler = StandardScaler()
    combined_features = scaler.fit_transform(combined_df[selected_features])

    # Use KMeans clustering
    num_clusters = max(int(1/2 * min(len(df_playlist1), len(df_playlist2))), 5)  # Ensure at least 5 clusters
    kmeans = KMeans(n_clusters=num_clusters, n_init=5, random_state=0)
    clusters = kmeans.fit_predict(combined_features)

    # Add cluster information to the dataframe
    combined_df['cluster'] = clusters

    # Update the original dataframes with the cluster information
    df_playlist1['cluster'] = clusters[:len(df_playlist1)]
    df_playlist2['cluster'] = clusters[len(df_playlist1):]
    a=len(df_playlist1)
    # Determine the minimum number of songs for the blended playlist
    if min_songs is None:
        min_songs = min(50,df_playlist1.shape[0],df_playlist2.shape[0]) # Set a reasonable default

    # Initialize the blended playlist
    blended_playlist = []

    # Select songs from clusters
    for c in np.unique(clusters):
        cluster_songs = combined_df[combined_df['cluster'] == c]

        # Separate the songs from each playlist within the cluster
        songs_playlist1 = cluster_songs.iloc[:sum(cluster_songs.index < len(df_playlist1))]
        songs_playlist2 = cluster_songs.iloc[sum(cluster_songs.index < len(df_playlist1)):]

        # Alternate between playlists for song selection
        while len(songs_playlist1) > 0 or len(songs_playlist2) > 0:
            if len(songs_playlist1) > 0:
                song = songs_playlist1.iloc[0]
                blended_playlist.append(song.to_dict())
                songs_playlist1 = songs_playlist1.iloc[1:]

            if len(songs_playlist2) > 0:
                song = songs_playlist2.iloc[0]
                blended_playlist.append(song.to_dict())
                songs_playlist2 = songs_playlist2.iloc[1:]

            if len(blended_playlist) >= min_songs:
                break

        if len(blended_playlist) >= min_songs:
            break

    # Calculate the similarity percentage
    num_common_clusters = len(np.intersect1d(df_playlist1['cluster'], df_playlist2['cluster']))
    total_clusters = len(np.union1d(df_playlist1['cluster'], df_playlist2['cluster']))
    similarity_percentage = (num_common_clusters / total_clusters) * 100
    df_blended_playlist = pd.DataFrame(blended_playlist)
    return df_blended_playlist, round(similarity_percentage)

@app.route('/')
def index():
    authorized = 'access_token' in session
    username = None
    if 'user_id' in session:
        user = User.query.get(session['user_id'])
        if user:
            username = user.username
    return render_template('index.html', authorized=authorized, username=username)



@app.route('/authorize')
def authorize():
    client_id = app.config['CLIENT_ID']
    redirect_uri = app.config['REDIRECT_URI']
    scope = app.config['SCOPE']
    state_key = os.urandom(15).hex()  # Creating a random state key
    session['state_key'] = state_key

    authorize_url = 'https://accounts.spotify.com/en/authorize?'
    params = {'response_type': 'code', 'client_id': client_id,
              'redirect_uri': redirect_uri, 'scope': scope,
              'state': state_key}
    query_params = urlencode(params)
    response = make_response(redirect(authorize_url + query_params))
    return response

@app.route('/callback/')
def callback():
    # Check for error
    error = request.args.get('error')
    if error:
        # Handle the error based on your requirements
        logging.error(f"Error during authorization: {error}")
        return render_template('error.html', error=error)

    # Check state to prevent CSRF
    state = request.args.get('state')
    if not state or state != session.get('state_key'):
        logging.error("State mismatch error")
        return render_template('error.html', error="State mismatch error")

    # Get the code from the callback URL
    code = request.args.get('code')

    # Use the code to get the access token
    access_token, refresh_token, expires_in = getToken(code)

    # Store the access token, refresh token, and expiration in the session or database
    session['access_token'] = access_token
    session['refresh_token'] = refresh_token
    session['expires_in'] = expires_in

    # Once access token is obtained, get user's Spotify display name
    headers = {'Authorization': f"Bearer {session['access_token']}"}
    response = requests.get("https://api.spotify.com/v1/me", headers=headers)
    if response.status_code == 200:
        spotify_data = response.json()
        spotify_id = spotify_data['id']  # Spotify ID to be used as primary key
        spotify_username = spotify_data['display_name']

    # Check if the user already exists in the database using the Spotify ID
        user = User.query.filter_by(id=spotify_id).first()
        if not user:
        # Create a new user with Spotify ID as the primary key
            user = User(id=spotify_id, username=spotify_username)
            db.session.add(user)
            db.session.commit()

    # Store the Spotify ID in the session
        session['user_id'] = spotify_id
        get_top_tracks(spotify_id, access_token, 50)
        get_user_playlists(access_token, spotify_id)
    return redirect(url_for('index'))
@app.route('/recommendations/', methods=['GET', 'POST'])
def get_recommendations():
    user_id = session.get('user_id')
    access_token = session.get('access_token')
    headers = {'Authorization': f"Bearer {access_token}"}
    if not access_token:
        return redirect(url_for('authorize'))

    if request.method == 'POST':
        input_data = request.form.get('custom_dataset')
      
        # Check if input is a URL or an ID
        if 'spotify.com' in input_data:
            # Extract the playlist ID from the URL
            match = re.search(r'playlist/([0-9A-Za-z?=_-]+)', input_data)
            playlist_id = match.group(1) if match else US_TOP_HITS_PLAYLIST_ID
        else:
            # Assume the input is a direct playlist ID
            playlist_id = input_data if input_data else US_TOP_HITS_PLAYLIST_ID
        # If a playlist ID is provided, use the playlist-based recommendation
        if playlist_id:
            playlist_features, playlist_tracks = fetch_playlist_features(headers,playlist_id)
        else:
            # Otherwise, use the top tracks for recommendation
            playlist_features, playlist_tracks = fetch_playlist_features(headers)
            
        track_ids = [track.id for track in Song.query.with_entities(Song.id).filter_by(uid=user_id, top_tracks=True).all()]
        track_characterstics=fetch_song_characteristics_batch(access_token,track_ids)
        tracks_df=pd.DataFrame(track_characterstics.values())
        cleaned_data = [item for item in playlist_features['audio_features'] if item is not None]
        us_top_hits_df = pd.DataFrame(cleaned_data)

        selected_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
        recommendations_df = get_k_nearest_songs(tracks_df, us_top_hits_df, selected_features, min_songs=10, max_songs=10)

        recommended_indices = recommendations_df.index.tolist()
        top_songs_info = []
        for i in recommended_indices:
            track = playlist_tracks['tracks']['items'][i]['track']
            track_name = track['name']
            track_artist = track['artists'][0]['name']
            track_url = track['external_urls']['spotify']
            duration_ms = track['duration_ms']
            duration_formatted = f"{duration_ms // 60000}:{(duration_ms % 60000) // 1000:02d}"
            top_songs_info.append({
                'name': track_name,
                'artist': track_artist,
                'url': track_url,
                'duration': duration_formatted
            })

        return render_template('recommended_tracks.html', tracks=top_songs_info)

    # Display the form to get playlist ID for recommendation
    return render_template('recommendations_form.html')

@app.route('/playlists/')
def display_playlists():
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))

    user_id = session.get('user_id')  # Get the user_id from the session
    if not user_id:
        # Handle the case where there is no user_id in the session
        return redirect(url_for('authorize'))  # Replace 'authorize' with your actual authorization route

    playlists = get_user_playlists(access_token, user_id)
    
    # Get all playlists for this user from the database (now they should be updated)
    user_playlists = Playlist.query.filter_by(user_id=user_id).all()
    #testing testing testing
    return render_template('display_playlists.html', playlists=user_playlists)


@app.route('/playlists/<playlist_id>/tracks/')
def display_tracks(playlist_id):
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))

    user_id = session.get('user_id')  # Get the user_id from the session
    if not user_id:
        # Handle the case where there is no user_id in the session
        return redirect(url_for('authorize'))  # Replace 'authorize' with your actual authorization route

    tracks = get_playlist_tracks(access_token, user_id, playlist_id)
    print(tracks)
    # Get all songs for this playlist from the database (now they should be updated)
    playlist_songs = db.session.query(
        Song.id, Song.name, Song.artists, playlist_song_association.c.added_on, Song.duration, Song.url
    ).join(
        playlist_song_association, (Song.id == playlist_song_association.c.song_id) & (Song.uid == playlist_song_association.c.user_id)
    ).filter(
        playlist_song_association.c.playlist_id == playlist_id, Song.uid == user_id
    ).all()
    print(playlist_songs)
    return render_template('display_tracks.html', tracks=tracks, playlist_songs=playlist_songs)



@app.route('/top_artists/')
def display_artists():
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))
    
    artists = get_top_artists(access_token, 50)
    all_genres = []

    user = session['user_id']

    for artist in artists:
        artist_name = artist['name']
        artist_id = artist['id']
        artist_image_url = artist['images'][0]['url'] if artist['images'] else None
        artist_genre = ', '.join(artist['genres']) if artist['genres'] else 'Unknown'
        all_genres.extend(artist['genres']) if artist['genres'] else None

        existing_artist = Top_Artists.query.filter_by(uid=user, id=artist_id).first()
        if existing_artist is None:
            top_artist = Top_Artists(uid=user, id=artist_id, name=artist_name, image_url=artist_image_url, genre=artist_genre)
            db.session.add(top_artist)
        else:
            existing_artist.image_url = artist_image_url
            existing_artist.genre = artist_genre
        db.session.commit()

    # Calculate the top 3 genres
    top_genres = [genre[0] for genre in Counter(all_genres).most_common(3)] if all_genres else ['Not available']

    top_artists = Top_Artists.query.filter_by(uid=user).all()

    return render_template('top_artists.html', artists=top_artists, top_genres=top_genres)


@app.route('/saved_albums/')
def display_saved_albums():
    #TESTING DON'T TOUCH
    access_token = session.get('access_token')
    if not access_token:
        return redirect(url_for('authorize'))

    saved_albums = get_saved_albums(access_token)

    for saved_album in saved_albums:
        for key, value in saved_album.items():
            if (key == 'album'):
                user = session['user_id']
                aname = value['name']
                album_id = value['uri']
                
                existing_album = Saved_Albums.query.filter_by(uid=user, aid=album_id).first()
                if existing_album is None:
                    salbum = Saved_Albums(uid=user, aid=album_id, name=aname)
                    db.session.add(salbum)
                    db.session.commit()

    saved_albums = Saved_Albums.query.filter_by(uid=user).all()

    return render_template('saved_albums.html', saved_albums = saved_albums)

@app.route('/top_tracks/')
def display_top_tracks():
    access_token = session.get('access_token')
    user_id = session.get('user_id')
    # Redirect to authorization if there's no access token
    if not access_token or not user_id:
        return redirect(url_for('authorize'))

    top_tracks = Song.query.filter_by(uid=user_id, top_tracks=True).all()
    # Render the top_tracks page and pass the retrieved top tracks to the template
    return render_template('top_tracks.html', top_tracks=top_tracks)

@app.route('/search', methods=['POST'])
def search():

    search_query = request.form.get('search-query')
    # Query the database for tracks with titles matching the search_query
    user = session['user_id']
    results = Song.query.filter(Song.name.like(f'{search_query}%'), Song.uid == user).all()
    return render_template('results.html', results=results)

@app.route('/blend_playlists', methods=['GET', 'POST'])
def blend_playlists_route():
    # Assuming you have a way to get the user's access token and user_id
    access_token = session.get('access_token')
    user_id = session.get('user_id')

    # Make sure the user is authenticated and we have the access token
    if access_token is None or user_id is None:
        return redirect(url_for('login'))  # Redirect to login if not authenticated

    if request.method == 'POST':
        # When the user submits the form, you will receive the two selected playlist IDs
        playlist_id1 = request.form.get('playlist1')
        playlist_id2 = request.form.get('playlist2')
        
        tracks_playlist1 = get_playlist_tracks(access_token, user_id, playlist_id1)
        tracks_playlist2 = get_playlist_tracks(access_token, user_id, playlist_id2)

        # Extract song IDs from both playlists
        song_ids_playlist1 = [track['id'] for track in tracks_playlist1]
        song_ids_playlist2 = [track['id'] for track in tracks_playlist2]
        all_song_ids = list(set(song_ids_playlist1 + song_ids_playlist2))  # Combine and remove duplicates

        # Fetch characteristics for all songs in batch
        all_songs_characteristics = fetch_song_characteristics_batch(access_token, all_song_ids)

        # Convert track data to dataframes
        df_playlist1_tracks = pd.DataFrame(tracks_playlist1)
        df_playlist2_tracks = pd.DataFrame(tracks_playlist2)

        # Create dataframes for characteristics using the batch data
        df_characteristics = pd.DataFrame(all_songs_characteristics.values())
        
        # Merge the track info with their characteristics
        df_playlist1 = pd.merge(df_playlist1_tracks, df_characteristics, left_on='id', right_on='id')
        df_playlist2 = pd.merge(df_playlist2_tracks, df_characteristics, left_on='id', right_on='id')
        # Define the features we are interested in for blending
        selected_features = [
            'danceability', 'energy', 'key', 'loudness',
            'mode', 'speechiness', 'acousticness', 'instrumentalness',
            'liveness', 'valence', 'tempo'
        ]

        # Use the blend_playlists function to find closest matches
        blended_playlist_df, compatibility = blend_playlists(df_playlist1, df_playlist2, selected_features)
        playlist1 = Playlist.query.filter_by(id=playlist_id1).first()
        playlist2 = Playlist.query.filter_by(id=playlist_id2).first()

        playlist_name1 = playlist1.name if playlist1 else f"Playlist 1"
        playlist_name2 = playlist2.name if playlist2 else f"Playlist 2"
        
# Construct the blend name using the playlist names
        blend_name = f"Blend between '{playlist_name1}' and '{playlist_name2}'"
        unique_blended_playlist_df = blended_playlist_df.drop_duplicates(subset='id')
        song_ids_str = ','.join(unique_blended_playlist_df['id'].tolist())
        blend_history_entry = BlendHistory(
            blend_name=blend_name,
            blender_id=user_id,
            blende_id=user_id,
            compatibility=compatibility,
            song_ids=song_ids_str,
            blend_type ="Playlist Blend"
        )
        db.session.add(blend_history_entry)
        db.session.commit()
        session['blended_tracks_ids'] = blended_playlist_df['id'].tolist()
          # Extract genres from the blended tracks
        genres_list = unique_blended_playlist_df['genres'].tolist()
        # Split genres by ', ' and flatten the list
        genres_flat_list = [genre for sublist in genres_list for genre in sublist.split(', ')]

        # Count the frequency of each genre
        genre_counts = Counter(genres_flat_list)

        # Find the most common genre
        top_genre = genre_counts.most_common(1)[0][0] if genre_counts else 'Not available'
        # Convert the blended tracks to a list of dictionaries to be easily rendered or used
        blended_tracks_list = unique_blended_playlist_df.to_dict('records')
        return render_template('show_blended_playlist.html', blended_tracks=blended_tracks_list, compatibility=compatibility,blend_id=blend_history_entry.blend_id,top_genre=top_genre)

    # If it's a GET request, show the playlists to the user to select which to blend
    user_playlists = get_user_playlists(access_token, user_id)
    return render_template('blend_playlists.html', playlists=user_playlists)
@app.route('/create_playlist', methods=['POST'])
def create_playlist_route():
    access_token = session.get('access_token')
    user_id = session.get('user_id')

    if access_token is None or user_id is None:
        return redirect(url_for('login'))

    playlist_name = request.form.get('playlist_name')

    if playlist_name:
        # Retrieve the track IDs from the form submission
        # 'track_selection' is the name attribute of the checkboxes
        selected_tracks = request.form.getlist('track_selection')

        if selected_tracks:
            playlist_id = create_spotify_playlist(access_token, user_id, playlist_name, selected_tracks)
            
            if playlist_id:
                playlist_url = get_playlist_url(access_token,playlist_id) # you need to implement this function
                blend_id = request.form.get('blend_id')  # make sure to send this from your form
                blend_history_entry = BlendHistory.query.get(blend_id)
                if blend_history_entry:
                    blend_history_entry.url = playlist_url  # Update the URL
                    blend_history_entry.blend_name = playlist_name  # Update the blend name if needed
                    db.session.commit()
    
                return render_template('playlist_created.html', playlist_id=playlist_id)
            else:
                return "Error creating playlist", 500
        else:
            return "No tracks selected", 400
    else:
        return "No playlist name provided", 400

# Helper function to create a new playlist and add tracks to it
def create_spotify_playlist(access_token, user_id, playlist_name, track_ids):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }

    payload = {
        'name': playlist_name,
        'description': 'New playlist description',
        'public': False  # or True, depending on your preference
    }

    response = requests.post(
        f'https://api.spotify.com/v1/users/{user_id}/playlists',
        headers=headers,
        json=payload
    )

    if response.status_code == 201:
        playlist_id = response.json()['id']
        success = add_tracks_to_playlist(access_token, playlist_id, track_ids)
        if not success:
            print('Error adding tracks to playlist')
        return playlist_id
    else:
        print(f'Error creating playlist: {response.json()}')
        return None

# Helper function to add tracks to the new playlist
def add_tracks_to_playlist(access_token, playlist_id, track_ids):
    headers = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}
    track_uris = [f'spotify:track:{track_id}' for track_id in track_ids]
    data = json.dumps({'uris': track_uris})
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    response = requests.post(url, headers=headers, data=data)
    
    return response.ok
def get_playlist_url(access_token, playlist_id):
    # Spotify endpoint to get playlist details
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

    # Headers for the GET request
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    # Make the GET request
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        playlist_data = response.json()
        # Return the external URL of the playlist
        return playlist_data.get('external_urls', {}).get('spotify')
    else:
        # Log error and return None if unsuccessful
        print(f"Error fetching playlist URL: {response.status_code}")
        return None

@app.route('/blend_users', methods=['GET', 'POST'])
def blend_users():
    current_user_id = session.get('user_id')
    access_token = session.get('access_token')

    if not access_token or not current_user_id:
        return redirect(url_for('login'))

    if request.method == 'POST':
        chosen_user_id = request.form.get('chosen_user_id')
        blend_type = request.form.get('blend_type', 'normal')
        reshuffle = request.form.get('reshuffle')

        if reshuffle:
            current_blend_type = "Reshuffle"
            chosen_user_id = session.get('chosen_user_id')
            fetch_all_tracks_for_user(current_user_id, access_token,session,True)
            fetch_all_tracks_for_user(chosen_user_id, access_token,session,True)
            current_user_tracks = Song.query.filter_by(uid=current_user_id).order_by(func.random()).limit(100).all()
            chosen_user_tracks = Song.query.filter_by(uid=chosen_user_id).order_by(func.random()).limit(100).all()
        elif blend_type == 'extensive':
            current_blend_type = "Extensive Blend"
            fetch_all_tracks_for_user(current_user_id, access_token,session)
            fetch_all_tracks_for_user(chosen_user_id, access_token,session)
            current_user_tracks = Song.query.filter_by(uid=current_user_id).limit(100).all()
            chosen_user_tracks = Song.query.filter_by(uid=chosen_user_id).limit(100).all()
        elif blend_type == 'genre':
            fetch_all_tracks_for_user(current_user_id, access_token,session)
            fetch_all_tracks_for_user(chosen_user_id, access_token,session)
            current_blend_type = "Genre Blend"
            genre_list = request.form.getlist('genres')  # Get the list of selected genres from the form

            genre_patterns = [f"%{genre}%" for genre in genre_list]
            current_user_tracks = Song.query.filter(
                Song.uid == current_user_id,
                or_(*[Song.genres.ilike(pattern) for pattern in genre_patterns])
            ).all()
            chosen_user_tracks = Song.query.filter(
                Song.uid == chosen_user_id,
                or_(*[Song.genres.ilike(pattern) for pattern in genre_patterns])
            ).all()
        else:
            current_blend_type = "Normal Blend"
            current_user_tracks = Song.query.filter_by(uid=current_user_id, top_tracks=True).all()
            chosen_user_tracks = Song.query.filter_by(uid=chosen_user_id, top_tracks=True).all()
        
        blender = User.query.get(current_user_id)
        blende = User.query.get(chosen_user_id)
        ownership = {}
        for track in current_user_tracks:
            ownership[track.id] = blender.username
        for track in chosen_user_tracks:
            if track.id in ownership:
                ownership[track.id] = 'both'
            else:
                ownership[track.id] = blende.username
        session['chosen_user_id'] = chosen_user_id
        current_user_song_ids = [track.id for track in current_user_tracks]
        chosen_user_song_ids = [track.id for track in chosen_user_tracks]

        current_user_tracks_features = fetch_song_characteristics_batch(access_token, current_user_song_ids)
        chosen_user_tracks_features = fetch_song_characteristics_batch(access_token, chosen_user_song_ids)

    # Convert the dictionaries to DataFrame
        df_current_user_tracks = pd.DataFrame.from_dict(current_user_tracks_features, orient='index')
        df_chosen_user_tracks = pd.DataFrame.from_dict(chosen_user_tracks_features, orient='index')


        
        selected_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

        blend_name = f"Blend between {blender.username} and {blende.username}"
    
        blended_tracks_df, score = blend_playlists(df_current_user_tracks, df_chosen_user_tracks, selected_features)
        unique_blended_playlist_df = blended_tracks_df.drop_duplicates(subset='id')
        song_ids_str = ','.join(unique_blended_playlist_df['id'].tolist())

        blend_history_entry = BlendHistory(
            blend_name=blend_name,
            blender_id=current_user_id,
            blende_id=chosen_user_id,
            compatibility=score,
            song_ids=song_ids_str,
            blend_type=current_blend_type 

        )

        db.session.add(blend_history_entry)
        db.session.commit()
        
        session['blended_tracks_ids'] = blended_tracks_df['id'].tolist()

        song_ids = blended_tracks_df['id'].tolist()

        songs_info = Song.query.filter(Song.id.in_(song_ids)).all()

        songs_info_df = pd.DataFrame([{
            'id': song.id,
            'name': song.name,
            'artists': song.artists,
            "genres":song.genres,
            'url': song.url
        } for song in songs_info])
        blended_tracks_complete_df = pd.merge(blended_tracks_df, songs_info_df, on='id')
        blended_tracks_unique_df = blended_tracks_complete_df.drop_duplicates(subset='id', keep='first')
        blended_tracks_list = blended_tracks_unique_df.to_dict('records')
        genre_counts = blended_tracks_complete_df['genres'].str.split(', ').explode().value_counts()
        top_genres = [genre for genre in genre_counts.head(3).index if genre]
        for track in blended_tracks_list:
            track['ownership'] = ownership.get(track['id'], 'unknown')
        # Calculating most shared artists
        # Extract artists from the current user's tracks
        current_user_artists = set()
        for track in current_user_tracks:
            current_user_artists.update(track.artists.split(', '))

# Extract artists from the chosen user's tracks
        chosen_user_artists = set()
        for track in chosen_user_tracks:
            chosen_user_artists.update(track.artists.split(', '))

# Find artists that appear in both users' tracks
        shared_artists = current_user_artists.intersection(chosen_user_artists)

# Count the frequency of each shared artist in the original track lists
        artist_counts = {}
        for track in current_user_tracks + chosen_user_tracks:
            for artist in track.artists.split(', '):
                if artist in shared_artists:
                    artist_counts[artist] = artist_counts.get(artist, 0) + 1

# Get top 5 shared artists based on their shared frequency
        top_shared_artists = sorted(artist_counts, key=artist_counts.get, reverse=True)[:5]

       
        return render_template('show_blended_playlist_user.html', blended_tracks=blended_tracks_list, compatibility=score, blend_id=blend_history_entry.blend_id, top_genres=top_genres,shared_artists=top_shared_artists)

    other_users = User.query.filter(User.id != current_user_id).all()
    return render_template('choose_user_to_blend.html', users=other_users)
@app.route('/view_blend_history')
def view_blend_history():
    current_user_id = session.get('user_id')
    if not current_user_id:
        return redirect(url_for('login'))

    # Fetch blend history for the current user
    user_blend_history = BlendHistory.query.filter(
        (BlendHistory.blender_id == current_user_id) | 
        (BlendHistory.blende_id == current_user_id)
    ).order_by(BlendHistory.blend_date.desc()).all()

    # Get unique user IDs from blend history
    user_ids = {blend.blender_id for blend in user_blend_history} | {blend.blende_id for blend in user_blend_history}

    # Fetch users and create a mapping of user ID to username
    users = User.query.filter(User.id.in_(user_ids)).all()
    username_mapping = {user.id: user.username for user in users}

    
    #stephanie testing blend
    blendees =[
        blend.blende_id if blend.blender_id == current_user_id else blend.blender_id
        for blend in user_blend_history
    ]

    from sqlalchemy import desc   
    blend_counts = (
        db.session.query(
            BlendHistory.blender_id,
            BlendHistory.blende_id,
            func.count().label('blend count')
        )
        .filter(
            BlendHistory.blend_type.in_(["Normal Blend", "Extensive Blend","Genre Blend"]),
            BlendHistory.blender_id == current_user_id,
            BlendHistory.blende_id.in_(blendees)
        )
        .group_by(BlendHistory.blender_id, BlendHistory.blende_id)
        .order_by(desc('blend count'))
        .first()
    )

    #top user information
    if blend_counts:
        # Extracting blender_id from blend_counts
        most_blended_user_id = blend_counts.blende_id
        top_count = getattr(blend_counts, 'blend count')

        # Joining blend_counts with User table to get the name of the most blended user
        most_blended_user = (
            db.session.query(User)
            .filter(User.id == most_blended_user_id)
            .first()
        )
        for blend in user_blend_history:
            blend.blender_username = username_mapping.get(blend.blender_id)
            blend.blende_username = username_mapping.get(blend.blende_id)
        return render_template('view_blend_history.html', blend_history=user_blend_history, blend_counts = blend_counts, most_blended_user = most_blended_user, top_count = top_count)
        


    
    for blend in user_blend_history:
        blend.blender_username = username_mapping.get(blend.blender_id)
        blend.blende_username = username_mapping.get(blend.blende_id)

    return render_template('view_blend_history.html', blend_history=user_blend_history, blend_counts = blend_counts)
@app.route('/view_songs_in_blend/<int:blend_id>')
def view_songs_in_blend(blend_id):
    blend = BlendHistory.query.get_or_404(blend_id)
    song_ids = blend.song_ids.split(',')  # Assuming song_ids are stored as a comma-separated string

    songs = Song.query.filter(Song.id.in_(song_ids)).distinct().all()

    return render_template('view_songs.html', songs=songs)

@app.route('/top_items')
def display_user():
    users = User.query.all()
    access_token = session.get('access_token')

    user_data = []

    for user in users:
        # user_id = user.id
        print(user.id)
        user_profile = get_user_profile(user.id, access_token)
        images_arr = user_profile['images'][0]['url'] if user_profile['images'] else None
        print(images_arr)
        user_data.append({"id": user.id, "name": user.username, "image_url": images_arr})

    print(user_data)


    return render_template('top_items.html', users=users, user_data = user_data)
@app.route('/search_blend', methods=['POST'])
def search_blend():
    search_query = request.form.get('search-query')
    current_user_id = session.get('user_id')

    if not current_user_id:
        # Handle the case when the user is not logged in
        return redirect(url_for('login'))

    # Assuming blend history includes both blender_id and blende_id
    blend_history = BlendHistory.query.filter(
        BlendHistory.blend_name.like(f"%{search_query}%"),
        (BlendHistory.blender_id == current_user_id) | 
        (BlendHistory.blende_id == current_user_id)
    ).all()

    # Render and return only the part of the page that needs to be updated
    return render_template('blend_history_list.html', blend_history=blend_history)

#stephanie testing do not touch
from sqlalchemy import func, and_

@app.route('/top_items/<user_username>')
def user_display(user_username):
    
    user = User.query.filter_by(username=user_username).first()
    print(user.username)
    print(user.id)

    if user:
        user_id = user.id 
        # get_top_tracks(user_id, access_token, 50)

        top_five_artists = (
            db.session.query(
                Top_Artists.name,
                func.row_number().over().label('artist_rank')
            )
            .join(User, User.id == Top_Artists.uid)
            .filter(User.id == user_id)
            .limit(5)
            .all()
        )

        top_five_songs = (
            db.session.query(
                Song.name,
                Song.artists,
                User.username,
                func.row_number().over(partition_by=User.id).label('song_rank')
            )
            .join(User, user.id == Song.uid)
            .filter(Song.top_tracks == True)
            .limit(5)
            .all()
        )

        # top_five_songs = (
        #     db.session.query(
        #         Song.name,
        #         Song.artists,
        #         User.username,
        #         func.row_number().over(partition_by=User.id).label('song_rank')
        #     )
        #     .join(User, User.id == Song.uid)
        #     .filter(Song.top_tracks == True)
        #     .limit(5)
        #     .all()
        # )

        for song, artist in zip(top_five_songs, top_five_artists):
            top_song_name = song[0]
            print(top_song_name)
            top_rank = artist[1]
            print(top_rank)
            top_song_artist = song[1]
            print(top_song_artist)
            top_artist_name = artist[0]
            print(top_artist_name)

            existing_record = User_Display.query.filter_by(uid=user_id, rank=top_rank).first()
            if existing_record is None:
                new_record = User_Display(
                    uid = user_id,
                    rank = top_rank,
                    song_name=top_song_name,
                    song_artist=top_song_artist,
                    artist=top_artist_name
                )
                db.session.add(new_record)
                db.session.commit()
        
        top_records = User_Display.query.filter_by(uid=user_id).all()
        genres_query = Top_Artists.query.with_entities(Top_Artists.genre).filter_by(uid=user_id).all()

        genres = [genre for (genre_list,) in genres_query for genre in genre_list.split(', ')]

        # Count the frequency of each genre
        genre_counts = Counter(genres)

        # Find the most common genre
        top_genre = genre_counts.most_common(3)

        public_playlists = Playlist.query.filter_by(user_id=user_id, public=True).all()
        print(public_playlists)




    return render_template('top_items_for_user.html', top_records=top_records, top_genre = top_genre, public_playlists = public_playlists)



if __name__ == "__main__":
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2023 11:43:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 11:43:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:43:39] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 11:43:39] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 11:43:40] "GET /authorize HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 11:43:40] "GET /authorize HTTP/1.1" 302 -
127.0.0.1 - - [09/Dec/2023 11:43:50] "GET /callback/?code=AQDmeg9T43PajN__8LVzLiXHxPmegFfUze8wLMFelIWQ1rCrbXogj-YBf6Iysr1EZRjjQDOSqvpNde-_tw6sjMvT9xHQbh1QvW0CsLIFVyLoRiWc_KvotqIVmo3YBdnk77Ozt2mTGL6jtjSkYRwpEa2twfrdhBIiut8rxgFTkRENZk2QlQ6JovzK8sIy3BGAXbxvogDjZvAyvVKwAosHeQ8QVFC4xRmpc9o

510d9feydv1cswxea5co7b9t2
https://i.scdn.co/image/ab67757000003b829ad00f249037ab6dba945463
ranjanjindal710


127.0.0.1 - - [09/Dec/2023 11:44:49] "GET /top_items HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 11:44:49] "GET /top_items HTTP/1.1" 200 -


None
[{'id': '510d9feydv1cswxea5co7b9t2', 'name': 'stephanie', 'image_url': 'https://i.scdn.co/image/ab67757000003b829ad00f249037ab6dba945463'}, {'id': 'ranjanjindal710', 'name': 'ranjanjindal710', 'image_url': None}]


...

stephanie
510d9feydv1cswxea5co7b9t2
Sunsetz
1
Cigarettes After Sex
Taylor Swift
愛你
2
Kimberley Chen
XG
Car's Outside
3
James Arthur
ABBA
Enchanted
4
Taylor Swift
Lee Young Ji
I Think I Like When It Rains
5
WILLIS
Ricky Montgomery
[<Playlist 6JlxB1yfPyludNWLQBXjFL>, <Playlist 2Uo6f2HCsSOKmoF799F3YK>, <Playlist 1M7tFdtVRNoGT5p3cNtmOC>, <Playlist 4iELDD151vga6wnq28vdEe>, <Playlist 3c1DGVP7xT5B5CiCv618Z0>, <Playlist 40KrI4BLmYMEo7zcOun0iu>, <Playlist 46Fu7BUj6AOSuaFnHTSyc9>, <Playlist 5MqP2tUvPqsoWWe38falFS>, <Playlist 13TGL4Qze67Ck9iD8rE79g>]


127.0.0.1 - - [09/Dec/2023 11:45:09] "GET /playlists/46Fu7BUj6AOSuaFnHTSyc9/tracks/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 11:45:09] "GET /playlists/46Fu7BUj6AOSuaFnHTSyc9/tracks/ HTTP/1.1" 200 -


[{'id': '0K2WjMLZYr09LKwurGRYRE', 'name': 'Theme From New York, New York - 2008 Remastered', 'artists': 'Frank Sinatra', 'genres': 'adult standards, easy listening, lounge', 'url': 'https://open.spotify.com/track/0K2WjMLZYr09LKwurGRYRE', 'duration': 204}, {'id': '0T0fkqReu2aO2H0VPUPWTo', 'name': "Baby I'm Yours - Digitally Remastered", 'artists': 'Cass Elliot', 'genres': '', 'url': 'https://open.spotify.com/track/0T0fkqReu2aO2H0VPUPWTo', 'duration': 152}, {'id': '1QlTNF43OyKK9BorIRsJ8o', 'name': 'Autumn Leaves', 'artists': 'Chet Baker', 'genres': 'cool jazz, jazz, jazz trumpet, vocal jazz', 'url': 'https://open.spotify.com/track/1QlTNF43OyKK9BorIRsJ8o', 'duration': 422}, {'id': '1bYItjcPRtxZs6Ir71O5RB', 'name': '(I Left My Heart) In San Francisco', 'artists': 'Tony Bennett', 'genres': 'adult standards, easy listening, lounge, vocal jazz', 'url': 'https://open.spotify.com/track/1bYItjcPRtxZs6Ir71O5RB', 'duration': 170}, {'id': '1ko2lVN0vKGUl9zrU0qSlT', 'name': 'Just the Two of Us (feat.

127.0.0.1 - - [09/Dec/2023 11:45:21] "GET /authorize HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 11:45:21] "GET /authorize HTTP/1.1" 302 -
127.0.0.1 - - [09/Dec/2023 11:45:22] "GET /callback/?code=AQDINo9Q_rzC9T-On37NuRlenDjLd1H8WkgQ3-9IZdtioI8iyCe4H7LW06aRvVsID7YdWVjjhbfZNdhpljvGm0tjuaXSs8vU8p9O82NmbPdsTncw0dFKJkP6afdDVZi1RW4a6Ir9G_yGSYiTmZw1clanvxP0_lg2nBAn3a4y-JZAlR03nGo8KFs-KE9d0lw5_XajVV_cc6vBh928ef6BhIQLEwCt7RXbYuJz4-br8uvr1SCGfoWMmP8DleZ2-TfUxG016tFWN6as2HucphCMLDe045caqmQctagWZGXme0ztaANGTcGA8mX4cUQf&state=23689811832763a5caebe833800fb7 HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 11:45:22] "GET /callback/?code=AQDINo9Q_rzC9T-On37NuRlenDjLd1H8WkgQ3-9IZdtioI8iyCe4H7LW06aRvVsID7YdWVjjhbfZNdhpljvGm0tjuaXSs8vU8p9O82NmbPdsTncw0dFKJkP6afdDVZi1RW4a6Ir9G_yGSYiTmZw1clanvxP0_lg2nBAn3a4y-JZAlR03nGo8KFs-KE9d0lw5_XajVV_cc6vBh928ef6BhIQLEwCt7RXbYuJz4-br8uvr1SCGfoWMmP8DleZ2-TfUxG016tFWN6as2HucphCMLDe045caqmQctagWZGXme0ztaANGTcGA8mX4cUQf&state=23689811832763a5caebe8338

stephanie
510d9feydv1cswxea5co7b9t2
Sunsetz
1
Cigarettes After Sex
Taylor Swift
愛你
2
Kimberley Chen
XG
Car's Outside
3
James Arthur
ABBA
Enchanted
4
Taylor Swift
Lee Young Ji
I Think I Like When It Rains
5
WILLIS
Ricky Montgomery
[<Playlist 6JlxB1yfPyludNWLQBXjFL>, <Playlist 2Uo6f2HCsSOKmoF799F3YK>, <Playlist 1M7tFdtVRNoGT5p3cNtmOC>, <Playlist 4iELDD151vga6wnq28vdEe>, <Playlist 3c1DGVP7xT5B5CiCv618Z0>, <Playlist 40KrI4BLmYMEo7zcOun0iu>, <Playlist 46Fu7BUj6AOSuaFnHTSyc9>, <Playlist 5MqP2tUvPqsoWWe38falFS>, <Playlist 13TGL4Qze67Ck9iD8rE79g>]


In [23]:
import sqlite3

# Replace with your actual database file path
database_path ='/home/cars327/csharp_final/csharp/db/User.db'
user_id = 'izkud41eox5ydye21ce50q9qw'  # Replace with the actual user ID

# Connect to the SQLite database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# SQL query that represents the same logic as the ORM query above
query = """
SELECT * FROM Song
WHERE uid = ? AND playlist_id = 'Top_Tracks'
"""

# Execute the query, passing the user_id as a parameter to prevent SQL injection
cursor.execute(query, (user_id,))

# Fetch all the resulting rows
top_tracks = cursor.fetchall()

# Print the rows, or process them as needed
for track in top_tracks:
    print(track)

# Close the connection
conn.close()



KeyboardInterrupt



In [2]:
pip install -U scikit-learn



  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/fe/6b/db949ed5ac367987b1f250f070f340b7715d22f0c9c965bdf07de6ca75a3/scikit_learn-1.3.2-cp312-cp312-win_amd64.whl.metadata
  Using cached scikit_learn-1.3.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.2-cp312-cp312-win_amd64.whl (9.1 MB)



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
